##### **Installing Libraries**

In [ ]:
!pip install talos # hyperparameter tuning
!pip install --upgrade pip

First of All I am going to Import **libraries** that will be usefull for this project

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # showing and rendering figures
# io related
import seaborn as sns # for Visualizing my data
from keras.utils import to_categorical # Using keras to_categorical because I need to convert by labels 
# into categorical form
import os # for taking input to my dataframe
import cv2 # for resizing my iamges
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))
# not needed in Kaggle, but required in Jupyter
%matplotlib inline 



In [ ]:
# Here I am building my dataframe (taking patient id  from image coloum and seperating side of image 
# left or right specefying path of image and at last converting my labels into categorical labels)
temp_df=pd.read_csv('../input/diabetic-retinopathy-detection/trainLabels.csv') # uploading csv to my pandas dataframe
print(temp_df.head()) # displaying first 5 objects in dataframe
image=temp_df['image'].str.split('_',n=1,expand=True) #splitting Side and Patient ID 
df = pd.DataFrame()# creating new dataframe object
df['eye_side']=image[1] #taking side of Image
df['patient_id']=image[0]#taking patient id of an Image

df['path']='../input/diabetic-retinopathy-detection/'#Giving paths of the images 
df['path']=df['path'].str.cat(temp_df['image']+'.jpeg')#adding Image path and format 
df['exists'] = df['path'].map(os.path.exists)
df=df[df['exists']]
df['level']=temp_df['level']# taking levels of Image
df['level_cat'] = df['level'].map(lambda x: to_categorical(x, 1+df['level'].max()))#converting my 
# labels to categorical_labels
df.head()

**Visualization**


In [ ]:
im = plt.imread(df.path.values[2]) # reading Image from its path

plt.imshow(im)# show the image
plt.show()

In [ ]:
sizes = df['level'].values #taking values from series because I only want to visualize levels nt index
print(sizes[0:5])#printing first 5 values


In [ ]:
sns.distplot(sizes, kde=False); # Visualizing levels in dataset

In [ ]:
pd.value_counts(sizes) # viewing the values of levels

In [ ]:
import PIL
from PIL import Image
baseheight = 128
img = Image.open('../input/diabetic-retinopathy-detection/1192_right.jpeg')

wsize = 128
img = img.resize((wsize, baseheight), PIL.Image.ANTIALIAS)
img.save('resized_image.jpg')# i need this for storing my previous image also 

In [ ]:
im = plt.imread('resized_image.jpg') # reading Image from its path

plt.imshow(im)# show the image
plt.title("Resized Image")
plt.show()

im = plt.imread('../input/diabetic-retinopathy-detection/1192_right.jpeg') # reading Image from its path

plt.imshow(im)# show the image
plt.title("Orignal Image")
plt.show()

In [ ]:
#total Examples of LEVEL [1,2,3,4] So that we are able to make balanced dataset
sum_E=0
for i in range (1,5):
    L1_df=pd.DataFrame()# creating new dataframe object
    L1_df =df [df.level==i]
    x=len(L1_df)
    sum_E=x+sum_E
print(sum_E)

So we will take **261** **examples from eye having** **level 0 and 261 examples from  level[1,2,3,4]**

**almost 70% of 0 DR and 30% of 1DR **

**Making of Binary dataframe**


B_df=pd.DataFrame()
#making secondary dataset having small number of sample 
#taking examples where level=0

L0_df=df[df.level==0 ]





L1_df=pd.DataFrame()# creating new dataframe object
L1_df=df[df.level==1 ]
L1_df=L1_df.drop('level_cat',axis=1)



L2_df=pd.DataFrame()# creating new dataframe object
L2_df=df[df.level==2 ]




L3_df=pd.DataFrame()# creating new dataframe object
L3_df=df[df.level==3] 
L3_df=L3_df.drop('level_cat',axis=1)



L4_df=pd.DataFrame()# creating new dataframe object
L4_df=df[df.level==4 ]
L4_df=L4_df.drop('level_cat',axis=1)


#Combining Examples of level [1,2,3,4]
frames = [ L1_df, L2_df,L3_df,L4_df]
L1_df = pd.concat(frames)
L1_df=L1_df.replace(({'level' : { 2 : 1, 3 : 1, 4 : 1 }}))


frames = [ L0_df,L1_df]
B_df = pd.concat(frames,sort=True)
B_df=B_df.drop('level_cat',axis=1)
B_df['level_cat'] = B_df['level'].map(lambda x: to_categorical(x, 1+B_df['level'].max()))#converting my 
# labels to categorical_labels
#Adding New Categorical Lables according to Binary Dataset

print("Total Numbe of examples in Binary DataFrame = " + str(len(B_df)))  # Total Examples

sizes =B_df['level'].values
sns.distplot(sizes, kde=False); # Visualizing levels in dataset

**Saving My Binary dataset and binary sizes (90,128,264) of images which are on same indexs **

**Saving Binary Dataset**

len(B_df)

B_df.to_csv("Binary Dataframe")


**Resizing image array to (90,90)**

#resizing my training examples to (90,90)
#train examples
from PIL import Image
from skimage.transform import resize 

image_list=[]

for i in range (len(B_df)):
    image=plt.imread(B_df['path'].iloc[i])
    image = resize(image, (90, 90))
    image_list.append(image)
    

**Resizing image array to (128,128)**

#resizing my training examples to (128,128)
#train examples
from PIL import Image
from skimage.transform import resize 

image_list_128=[]
for i in B_df['path']:
    image=plt.imread(i)
    image = resize(image, (128, 128))
    image_list_128.append(image)

**Resizing image array to (264,264)**

#resizing my training examples to (264,264)
#train examples
from PIL import Image
from skimage.transform import resize 

image_list_264=[]
for i in B_df['path']:
    image=plt.imread(i)
    image = resize(image, (264, 264))
    image_list_264.append(image)

**Only resizing took my 3:00 hours in Kaggle** So I am storing my preprocess data

**Saving Numpy array to files**

img_list=np.asarray(image_list)
img_list=img_list.reshape(1000,90*90*3)
print(img_list.shape)
type(img_list)
np.savez_compressed('1000_Binary_images_data_90', a=img_list)

img_list_128=np.asarray(image_list_128)
img_list_128=img_list_128.reshape(1000,128*128*3)
print(img_list_128.shape)
type(img_list_128)
np.savez_compressed('1000_Binary_images_data_128', a=img_list_128)

img_list_264=np.asarray(image_list_264)
img_list_264=img_list_264.reshape(1000,264*264*3)
print(img_list_264.shape)
type(img_list_264)
np.savez_compressed('1000_Binary_images_data_264', a=img_list_264)

**loading my saving Binary data**

In [ ]:
B_df=pd.read_csv('../input/prepossessed-arrays-of-binary-data/1000_Binary Dataframe')
B_df=B_df.drop('Unnamed: 0',axis=1)
B_df.head(10)

In [ ]:
sizes =B_df['level'].values
sns.distplot(sizes, kde=False); # Visualizing levels in dataset

**loading My numpy array which I saved**

In [ ]:
Binary_90 = np.load('../input/prepossessed-arrays-of-binary-data/1000_Binary_images_data_90.npz')
X_90=Binary_90['a']
Binary_128 = np.load('../input/prepossessed-arrays-of-binary-data/1000_Binary_images_data_128.npz')
X_128=Binary_128['a']
Binary_264 = np.load('../input/prepossessed-arrays-of-binary-data/1000_Binary_images_data_264.npz')
X_264=Binary_264['a']
y=B_df['level'].values


print(X_90.shape)
print(X_128.shape)
print(X_264.shape)
print(y.shape)

In [ ]:
# we need to resize our X because we load array in 2 diminsional and we need it in 4 diminsional
print("Shape before reshaping X_90" +str(X_90.shape))
X_90=X_90.reshape(1000,90,90,3)
print("Shape after reshaping X_90" +str(X_90.shape))
print("\n\n")

print("Shape before reshaping X_128" +str(X_128.shape))
X_128=X_128.reshape(1000,128,128,3)
print("Shape after reshaping X_128" +str(X_128.shape))
print("\n\n")

print("Shape before reshaping X_264" +str(X_264.shape))
X_264=X_264.reshape(1000,264,264,3)
print("Shape after reshaping X_264" +str(X_264.shape))


**Confirming that image path on 1st index of data frame is same as the image at 1st index of the Np Arrays**

In [ ]:
im = plt.imread(B_df['path'][1]) # reading Image from its path

plt.imshow(im)# show the image
plt.title("Orignal Image")
plt.show()

In [ ]:
plt.title("90*90*3 Image")
plt.imshow(X_90[1])
plt.show()

plt.title("128*128*3 Image")
plt.imshow(X_128[1])
plt.show()

plt.title("264*264*3 Image")
plt.imshow(X_264[1])
plt.show()

**Its seem to be same now thats good for us**

**Now I have my X and Y. Now its time for spliting and training**

In [ ]:
y.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_128,y, test_size=0.10, random_state=42)
y_train = to_categorical(y_train, num_classes=2)
y_test_Categorical=to_categorical(y_test)
y_categorical =to_categorical(y)

**CNN MODEL **

In [ ]:
from keras.models import Sequential,Model
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout,Activation
from keras import losses
from keras.optimizers import Adam, Adagrad
from keras.callbacks import EarlyStopping
from keras import regularizers
from sklearn.model_selection import GridSearchCV
import keras
#import talos as ta

**Talos Model For HyperParameter Optimization**

**Talos for hyperparameter tuning**

In [ ]:

def Talos_Model(X_train, y_train, X_test, y_test, params):
    #parameters defined
    lr = params['lr']
    epochs=params['epochs']
    dropout_rate=params['dropout']
    optimizer=params['optimizer']
    loss=params['loss']
    last_activation=params['last_activation']
    activation=params['activation']
    clipnorm=params['clipnorm']
    decay=params['decay']
    momentum=params['momentum']
    l1=params['l1']
    l2=params['l2']
    No_of_CONV_and_Maxpool_layers=params['No_of_CONV_and_Maxpool_layers']
    No_of_Dense_Layers =params['No_of_Dense_Layers']
    No_of_Units_in_dense_layers=params['No_of_Units_in_dense_layers']
    Kernal_Size=params['Kernal_Size']
    Conv2d_filters=params['Conv2d_filters']
    pool_size_p=params['pool_size']
    padding_p=params['padding']
    
    #model sequential
    model=Sequential()
    
    for i in range(0,No_of_CONV_and_Maxpool_layers):
        model.add(Conv2D(Conv2d_filters, Kernal_Size ,padding=padding_p))
        model.add(Activation(activation))
        model.add(MaxPooling2D(pool_size=pool_size_p,strides=(2,2)))
    
    
    model.add(Flatten())
    
    for i in range (0,No_of_Dense_Layers):
        model.add(Dense(units=No_of_Units_in_dense_layers,activation=activation, kernel_regularizer=regularizers.l2(l2),
                  activity_regularizer=regularizers.l1(l1)))
    
    
    model.add(Dense(units=20,activation=activation))
    
    model.add(Dense(units=2,activation=activation))
    if optimizer=="Adam":
        opt=keras.optimizers.Adam(lr=lr, decay=decay, beta_1=0.9, beta_2=0.999)
    if optimizer=="Adagrad":
        opt=keras.optimizers.Adagrad(lr=lr, epsilon=None, decay=decay)
    if optimizer=="sgd":
        opt=keras.optimizers.SGD(lr=lr, momentum=momentum, decay=decay, nesterov=False)
    
    model.compile(loss=loss,optimizer=opt,
                 metrics=['accuracy'])
    
    out = model.fit(X_train, y_train, epochs=params['epochs'])

    return out,model

**Hyperparameter Tuning  by using Talos**

[https://github.com/autonomio/talos/blob/master/examples/Hyperparameter%20Optimization%20with%20Keras%20for%20the%20Iris%20Prediction.ipynb](http://)

In [ ]:


params = {'lr': (0.1, 0.01,1 ),
     'epochs': [10,5,15],
     'dropout': (0, 0.40, 0.8),
     'optimizer': ["Adam","Adagrad","sgd"],
     'loss': ["binary_crossentropy","mean_squared_error","mean_absolute_error"],
     'last_activation': ["softmax","sigmoid"],
     'activation' :["relu","selu","linear"],
     'clipnorm':(0.0,0.5,1),
     'decay':(1e-6,1e-4,1e-2),
     'momentum':(0.9,0.5,0.2),
     'l1': (0.01,0.001,0.0001),
     'l2': (0.01,0.001,0.0001),
     'No_of_CONV_and_Maxpool_layers':[2,3],
     'No_of_Dense_Layers': [2,3,4],
     'No_of_Units_in_dense_layers':[128,64,32,256],
     'Kernal_Size':[(2,2),(4,4),(6,6)],
     'Conv2d_filters':[60,40,80,120],
     'pool_size':[(2,2),(4,4)],
     'padding':["valid","same"]
    }


Talos take 1500 different scans which is so much expansive even my kaggle kernal die because of talos scans 

In [ ]:
import talos as ta
h = ta.Scan(X_train, y_train, params=params,
            model=Talos_Model,
            dataset_name='DR',
            experiment_no='1',
            grid_downsample=.01)

**Reporting of talos library**

In [ ]:
accessing the results data frame
h.data.head()

accessing epoch entropy values for each round
h.peak_epochs_df

access the summary details
h.details

In [ ]:
r = ta.Reporting(h)

r.best_params()



**Randomized Search For Hyperparameter tuning**

**Model**

In [ ]:

def Randomized_Model(lr=0.01,dropout=0.5,optimizer="adam",loss='mean_squared_error',
                    last_activation="softmax",activation="relu",clipnorm=0.1,
                    decay=1e-2,momentum=0.5,l1=0.01,l2=0.001,No_of_CONV_and_Maxpool_layers=3,
                    No_of_Dense_Layers=3,No_of_Units_in_dense_layers=24,Conv2d_filters=60):
       
    
    
    #model sequential
    model=Sequential()
    
    for i in range(0,No_of_CONV_and_Maxpool_layers):
        model.add(Conv2D(Conv2d_filters, (2,2) ,padding="same"))
        model.add(Activation(activation))
        model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    
    
    model.add(Flatten())
    
    for i in range (0,No_of_Dense_Layers):
        model.add(Dense(units=No_of_Units_in_dense_layers,activation=activation, kernel_regularizer=regularizers.l2(l2),
                  activity_regularizer=regularizers.l1(l1)))
    
    model.add(Dropout(dropout))
    model.add(Dense(units=20,activation=activation))
    
    model.add(Dense(units=2,activation=activation))
    if optimizer=="Adam":
        opt=keras.optimizers.Adam(lr=lr, decay=decay, beta_1=0.9, beta_2=0.999)
    if optimizer=="Adagrad":
        opt=keras.optimizers.Adagrad(lr=lr, epsilon=None, decay=decay)
    if optimizer=="sgd":
        opt=keras.optimizers.SGD(lr=lr, momentum=momentum, decay=decay, nesterov=False)
    
    model.compile(loss=loss,optimizer=opt,
                 metrics=['accuracy'])
    
    

    return model

**Parameters of Randomized Search**

In [ ]:


params = {'lr': (0.1, 0.01,1,0.001 ),
     'epochs': [10,5,15,30],
     'dropout': (0, 0.40, 0.8),
     'optimizer': ["Adam","Adagrad","sgd"],
     'loss': ["binary_crossentropy","mean_squared_error","mean_absolute_error"],
     'last_activation': ["softmax","sigmoid"],
     'activation' :["relu","selu","linear"],
     'clipnorm':(0.0,0.5,1),
     'decay':(1e-6,1e-4,1e-2),
     'momentum':(0.9,0.5,0.2),
     'l1': (0.01,0.001,0.0001),
     'l2': (0.01,0.001,0.0001),
     'No_of_CONV_and_Maxpool_layers':[2,3],
     'No_of_Dense_Layers': [2,3,4,5],
     'No_of_Units_in_dense_layers':[128,64,32,256],
     
     'Conv2d_filters':[60,40,80,120,220]
     
     
    }


In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import make_scorer
# model class to use in the scikit random search CV 
model = KerasClassifier(build_fn=Randomized_Model, epochs=10, batch_size=20, verbose=1)
grid = RandomizedSearchCV(estimator=model, cv=KFold(3), param_distributions=params, 
                          verbose=20,  n_iter=10, n_jobs=1)


In [ ]:
grid_result = grid.fit(X_train, y_train)

**Best Params of Randomized search**

In [ ]:
best_params=grid_result.best_params_
best_params

In [ ]:

from sklearn.metrics import accuracy_score

y=grid_result.predict(X_test)
random=accuracy_score(y, y_test)
print("Base Accuracy ",random)

best_random = grid_result.best_estimator_
y1=best_random.predict(X_test)
Best=accuracy_score(y1, y_test)
print("Best Accuracy " ,Best)


print('Improvement of {:0.2f}%.'.format( 100 * (Best - random) / random))

In [ ]:
def Best_param_Model(best_params):
       
    lr=best_params["lr"]
    dropout=best_params["dropout"]
    optimizer=best_params["optimizer"]
    loss=best_params["loss"]
    last_activation=best_params["last_activation"]
    activation=best_params["activation"]
    clipnorm=best_params["clipnorm"]
    decay=best_params["decay"]
    momentum=best_params["momentum"]
    l1=best_params["l1"]
    l2=best_params["l2"]
    No_of_CONV_and_Maxpool_layers=best_params["No_of_CONV_and_Maxpool_layers"]
    No_of_Dense_Layers=best_params["No_of_Dense_Layers"]
    No_of_Units_in_dense_layers=best_params["No_of_Units_in_dense_layers"]
    Conv2d_filters=best_params["Conv2d_filters"]
    
    #model sequential
    model=Sequential()
    
    for i in range(0,No_of_CONV_and_Maxpool_layers):
        model.add(Conv2D(Conv2d_filters, (2,2) ,padding="same"))
        model.add(Activation(activation))
        model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    
    
    model.add(Flatten())
    
    for i in range (0,No_of_Dense_Layers):
        model.add(Dense(units=No_of_Units_in_dense_layers,activation=activation, kernel_regularizer=regularizers.l2(l2),
                  activity_regularizer=regularizers.l1(l1)))
    
    
    model.add(Dense(units=20,activation=activation))
    
    model.add(Dense(units=2,activation=activation))
    if optimizer=="Adam":
        opt=keras.optimizers.Adam(lr=lr, decay=decay, beta_1=0.9, beta_2=0.999)
    if optimizer=="Adagrad":
        opt=keras.optimizers.Adagrad(lr=lr, epsilon=None, decay=decay)
    if optimizer=="sgd":
        opt=keras.optimizers.SGD(lr=lr, momentum=momentum, decay=decay, nesterov=False)
    
    model.compile(loss=loss,optimizer=opt,
                 metrics=['accuracy'])
    
    

    return model

In [ ]:

Binary_model=Best_param_Model(best_params)
history =Binary_model.fit(X_train, y_train, epochs=100, validation_data=(X_test,y_test_Categorical))

# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])


plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
Binary_model.evaluate(X_test,y_test_Categorical)

**Visualizing my CNN model**

In [ ]:
y=B_df['level'].values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_128,y, test_size=0.10, random_state=42)
y_train = to_categorical(y_train, num_classes=2)
y_test_Categorical=to_categorical(y_test)

In [ ]:

model = Sequential()
model.add(Conv2D(16,kernel_size = (5,5),activation = 'relu', activity_regularizer=regularizers.l2(1e-8)))
model.add(Conv2D(32,kernel_size = (5,5),activation = 'relu', activity_regularizer = regularizers.l2(1e-8)))
model.add(MaxPooling2D(3,3))
model.add(Conv2D(64,kernel_size = (5,5),activation = 'relu', activity_regularizer = regularizers.l2(1e-8)))
model.add(MaxPooling2D(3,3))
model.add(Conv2D(128,activation = 'relu',kernel_size = (3,3),activity_regularizer = regularizers.l2(1e-8)))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(64,activation = 'tanh',activity_regularizer = regularizers.l2(1e-8)))
model.add(Dropout(0.2))
model.add(Dense(16,activation = 'tanh',activity_regularizer = regularizers.l2(1e-8)))
model.add(Dropout(0.2))
model.add(Dense(2,activation = 'softmax'))
model.compile(loss=keras.losses.binary_crossentropy, optimizer="adam", metrics=["accuracy"])
model.fit(X_train,y_train, epochs = 10 ,batch_size = 16,validation_data=(X_test,y_test_Categorical))
model.summary()


Accuracy Measures
[Helping Material](http://comprna.upf.edu/courses/Master_AGB/2_ClassificationAlgorithms/Lecture_Accuracy.pdf)

In [ ]:
from sklearn.metrics import confusion_matrix
prediction=model.predict(X_test)
y_pred=[]
for i in prediction:
    y_pred.append(i.argmax())
y_pred=np.asarray(y_pred)
true_negative,false_positive,false_negative,true_positive=confusion_matrix(y_test, y_pred).ravel()

print("true_negative: ",true_negative)
print("false_positive: ",false_positive)
print("false_negative: ",false_negative)
print("true_positive: ",true_positive)
print("\n\n Accuracy Measures\n\n")
Sensitivity=true_positive/(true_positive+false_negative)
print("Sensitivity: ",Sensitivity)

False_Positive_Rate=false_positive/(false_positive+true_negative)
print("False_Positive_Rate: ",False_Positive_Rate)

Specificity=true_negative/(false_positive + true_negative)
print("Specificity: ",Specificity)

#FDR à 0 means that very few of our predictions are wrong
False_Discovery_Rate=false_positive/(false_positive+true_positive)
print("False_Discovery_Rate: ",False_Discovery_Rate)

Positive_Predictive_Value =true_positive/(true_positive+false_positive)
print("Positive_Predictive_Value: ",Positive_Predictive_Value)



**Model is Giving 0% accuracy After all of this If someone know the solution please let me know**
1. According to me less images will be the problem so that cnn is not getting enough images to train their weights well

In [ ]:
a=np.expand_dims( X_train[10],axis=0)
a.shape
layer_outputs = [layer.output for layer in model.layers]
activation_model = Model(inputs=model.input, outputs=layer_outputs)
activations = activation_model.predict(a)


In [ ]:
def display_activation(activations, col_size, row_size, act_index): 
    activation = activations[act_index]
    activation_index=0
    fig, ax = plt.subplots(row_size, col_size, figsize=(row_size*2.5,col_size*1.5))
    for row in range(0,row_size):
        for col in range(0,col_size):
            ax[row][col].imshow(activation[0, :, :, activation_index])
            activation_index += 1
display_activation(activations, 4, 4,1)

please visualize 1 and 3 shape their will be a 6d array of filters but first 16 filter not present in the weights array

**Visualizing 15th filter of my first cov2d layer**

In [ ]:
top_layer = model.layers[0]
plt.imshow(top_layer.get_weights()[0][:, :, :,15 ])

**Achiving 76% accuracy till now on 1000 examples**